In [1]:
# coding=utf-8
"""
This module contains the tests for the exchanges classes
"""
from fastlane_bot import Bot, Config
from fastlane_bot.bot import CarbonBot
from fastlane_bot.tools.cpc import ConstantProductCurve
from fastlane_bot.tools.cpc import ConstantProductCurve as CPC
from fastlane_bot.events.exchanges import UniswapV2, UniswapV3, SushiswapV2, CarbonV1, BancorV3
from fastlane_bot.events.interface import QueryInterface
from fastlane_bot.helpers.poolandtokens import PoolAndTokens
from fastlane_bot.helpers import TradeInstruction, TxReceiptHandler, TxRouteHandler, TxSubmitHandler, TxHelpers, TxHelper
from fastlane_bot.events.managers.manager import Manager
from fastlane_bot.events.interface import QueryInterface
from joblib import Parallel, delayed
import pytest
import math
import json
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(Bot))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(UniswapV3))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(SushiswapV2))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CarbonV1))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(BancorV3))
from fastlane_bot.testing import *
from fastlane_bot.modes import triangle_single_bancor3
#plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
from fastlane_bot import __VERSION__
require("3.0", __VERSION__)

ConstantProductCurve v2.14 (23/May/2023)
CarbonBot v3-b2.2 (20/June/2023)
UniswapV2 v0.0.1 (2023-07-03)
UniswapV3 v0.0.1 (2023-07-03)
SushiswapV2 v0.0.1 (2023-07-03)
CarbonV1 v0.0.1 (2023-07-03)
BancorV3 v0.0.1 (2023-07-03)
imported m, np, pd, plt, os, sys, decimal; defined iseq, raises, require
Version = 3-b2.2 [requirements >= 3.0 is met]


# Single Mode [NB038]

In [2]:
C = cfg = Config.new(config=Config.CONFIG_MAINNET)
C.DEFAULT_MIN_PROFIT_BNT = 0.02
C.DEFAULT_MIN_PROFIT = 0.02
cfg.DEFAULT_MIN_PROFIT_BNT = 0.02
cfg.DEFAULT_MIN_PROFIT = 0.02
assert (C.NETWORK == C.NETWORK_MAINNET)
assert (C.PROVIDER == C.PROVIDER_ALCHEMY)
setup_bot = CarbonBot(ConfigObj=C)
pools = None
with open('fastlane_bot/data/tests/latest_pool_data_testing.json') as f:
    pools = json.load(f)
pools = [pool for pool in pools]
pools[0]
static_pools = pools
state = pools
exchanges = list({ex['exchange_name'] for ex in state})
db = QueryInterface(state=state, ConfigObj=C, exchanges=exchanges)
setup_bot.db = db

static_pool_data_filename = "static_pool_data"

static_pool_data = pd.read_csv(f"fastlane_bot/data/{static_pool_data_filename}.csv", low_memory=False)
    
uniswap_v2_event_mappings = pd.read_csv("fastlane_bot/data/uniswap_v2_event_mappings.csv", low_memory=False)
        
tokens = pd.read_csv("fastlane_bot/data/tokens.csv", low_memory=False)
        
exchanges = "carbon_v1,bancor_v3,uniswap_v3,uniswap_v2,sushiswap_v2"

exchanges = exchanges.split(",")


alchemy_max_block_fetch = 20
static_pool_data["cid"] = [
        cfg.w3.keccak(text=f"{row['descr']}").hex()
        for index, row in static_pool_data.iterrows()
    ]
# Filter out pools that are not in the supported exchanges
static_pool_data = [
    row for index, row in static_pool_data.iterrows()
    if row["exchange_name"] in exchanges
]

static_pool_data = pd.DataFrame(static_pool_data)
static_pool_data['exchange_name'].unique()
# Initialize data fetch manager
mgr = Manager(
    web3=cfg.w3,
    cfg=cfg,
    pool_data=static_pool_data.to_dict(orient="records"),
    SUPPORTED_EXCHANGES=exchanges,
    alchemy_max_block_fetch=alchemy_max_block_fetch,
    uniswap_v2_event_mappings=uniswap_v2_event_mappings,
    tokens=tokens.to_dict(orient="records"),
)

# Add initial pools for each row in the static_pool_data
start_time = time.time()
Parallel(n_jobs=-1, backend="threading")(
    delayed(mgr.add_pool_to_exchange)(row) for row in mgr.pool_data
)
cfg.logger.info(f"Time taken to add initial pools: {time.time() - start_time}")

# check if any duplicate cid's exist in the pool data
mgr.deduplicate_pool_data()
cids = [pool["cid"] for pool in mgr.pool_data]
assert len(cids) == len(set(cids)), "duplicate cid's exist in the pool data"
def init_bot(mgr: Manager) -> CarbonBot:
    """
    Initializes the bot.

    Parameters
    ----------
    mgr : Manager
        The manager object.

    Returns
    -------
    CarbonBot
        The bot object.
    """
    mgr.cfg.logger.info("Initializing the bot...")
    bot = CarbonBot(ConfigObj=mgr.cfg)
    bot.db = db
    bot.db.mgr = mgr
    assert isinstance(
        bot.db, QueryInterface
    ), "QueryInterface not initialized correctly"
    return bot
bot = init_bot(mgr)
# add data cleanup steps from main.py
bot.db.handle_token_key_cleanup()
bot.db.remove_unmapped_uniswap_v2_pools()
bot.db.remove_zero_liquidity_pools()
bot.db.remove_unsupported_exchanges()
tokens = bot.db.get_tokens()
ADDRDEC = {t.key: (t.address, int(t.decimals)) for t in tokens if not math.isnan(t.decimals)}
flashloan_tokens = bot.setup_flashloan_tokens(None)
CCm = bot.setup_CCm(None)
pools = db.get_pool_data_with_tokens()

arb_mode = "single"

2023-07-25 23:09:05,731 [fastlane:INFO] - Time taken to add initial pools: 0.14443588256835938
2023-07-25 23:09:05,736 [fastlane:INFO] - Initializing the bot...
2023-07-25 23:09:05,876 [fastlane:INFO] - Removed 3242 unmapped uniswap_v2/sushi pools. 1897 uniswap_v2/sushi pools remaining
2023-07-25 23:09:05,876 [fastlane:INFO] - Unmapped uniswap_v2/sushi pools:
2023-07-25 23:09:06,183 [fastlane:INFO] - uniswap_v3: 0
2023-07-25 23:09:06,184 [fastlane:INFO] - uniswap_v2: 3242
2023-07-25 23:09:06,184 [fastlane:INFO] - sushiswap_v2: 0
2023-07-25 23:09:06,186 [fastlane:INFO] - uniswap_v3: 636
2023-07-25 23:09:06,187 [fastlane:INFO] - sushiswap_v2: 78
2023-07-25 23:09:06,187 [fastlane:INFO] - uniswap_v2: 0
2023-07-25 23:09:06,188 [fastlane:INFO] - bancor_v2: 0
2023-07-25 23:09:06,188 [fastlane:INFO] - bancor_v3: 34
2023-07-25 23:09:06,188 [fastlane:INFO] - carbon_v1: 220
2023-07-25 23:09:06,235 [fastlane:INFO] - uniswap_v3_zero_liquidity_pools: 858
2023-07-25 23:09:06,235 [fastlane:INFO] - sus

In [3]:
assert(cfg.DEFAULT_MIN_PROFIT_BNT <= 0.02), f"[TestSingleMode], DEFAULT_MIN_PROFIT_BNT must be <= 0.02 for this Notebook to run, currently set to {cfg.DEFAULT_MIN_PROFIT_BNT}"
assert(C.DEFAULT_MIN_PROFIT_BNT <= 0.02), f"[TestSingleMode], DEFAULT_MIN_PROFIT_BNT must be <= 0.02 for this Notebook to run, currently set to {cfg.DEFAULT_MIN_PROFIT_BNT}"

## Test_arb_mode_class

In [4]:
arb_finder = bot._get_arb_finder("single")
assert arb_finder.__name__ == "FindArbitrageSinglePairwise", f"[TestSingleMode] Expected arb_finder class name name = FindArbitrageSinglePairwise, found {arb_finder.__name__}"

## Test_tokens_and_combos

In [5]:
arb_finder = bot._get_arb_finder("single")
finder2 = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_TOKENS,
            ConfigObj=bot.ConfigObj,
        )
all_tokens, combos = finder2.find_arbitrage()
assert len(all_tokens) == 545, f"[TestMultiMode] Using wrong dataset, expected 545 tokens, found {len(all_tokens)}"
assert len(combos) == 3264, f"[TestMultiMode] Using wrong dataset, expected 3264 tokens, found {len(combos)}"

### Test_Single_Arb_Finder_vs_run

In [6]:
run_full = bot._run(flashloan_tokens=flashloan_tokens, CCm=CCm, arb_mode=arb_mode, data_validator=False, result=bot.XS_ARBOPPS)
arb_finder = bot._get_arb_finder("single")
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()
assert len(r) == 22, f"[TestSingleMode] Expected 22 arbs, found {len(r)}"
assert len(r) == len(run_full), f"[TestSingleMode] Expected arbs from .find_arbitrage - {len(r)} - to match _run - {len(run_full)}"

  0%|          | 0/1 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]


  0%|          | 0/1 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]


  0%|          | 0/2 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]


  0%|          | 0/4 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]


In [7]:
r

[(6.711868213631777,
                                                      WETH-6Cc2      HEX-eb39
  0x6d2372776a5beac5ae5352308a74201337ea626ff1a94...   0.002687 -5.792822e+02
  14291859410679415465461733512134264881242-0         -0.004096  5.792822e+02
  PRICE                                                1.000000  4.639073e-06
  AMMIn                                                0.002687  5.792822e+02
  AMMOut                                              -0.004096 -5.792822e+02
  TOTAL NET                                           -0.001409  5.474997e-07,
  ({'cid': '0x6d2372776a5beac5ae5352308a74201337ea626ff1a949243eb051481792aa2f',
    'tknin': 'WETH-6Cc2',
    'amtin': 0.002687311916758972,
    'tknout': 'HEX-eb39',
    'amtout': -579.2821683883667,
    'error': None},
   {'cid': '14291859410679415465461733512134264881242-0',
    'tknin': 'HEX-eb39',
    'amtin': 579.2821689358664,
    'tknout': 'WETH-6Cc2',
    'amtout': -0.00409614349875,
    'error': None}),
  'WETH-6Cc2',

## Test_no_multi_carbon

In [8]:
arb_finder = bot._get_arb_finder("single")
finder = arb_finder(
            flashloan_tokens=flashloan_tokens,
            CCm=CCm,
            mode="bothin",
            result=bot.AO_CANDIDATES,
            ConfigObj=bot.ConfigObj,
        )
r = finder.find_arbitrage()
multi_carbon_count = 0

for arb in r:
    (
            best_profit,
            best_trade_instructions_df,
            best_trade_instructions_dic,
            best_src_token,
            best_trade_instructions,
        ) = arb
    if len(best_trade_instructions_dic) > 2:
        multi_carbon_count += 1

assert multi_carbon_count == 0, f"[TestSingleMode] Expected arbs without multiple Carbon curves, but found {len(multi_carbon_count)}"

  0%|          | 0/1 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]


  0%|          | 0/1 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]


  0%|          | 0/2 [00:00<?, ?it/s]

...

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]


  0%|          | 0/4 [00:00<?, ?it/s]

[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
[TradeInstruction.to_dicts]
